# Model to prediction AGE 

In [89]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split


In [90]:
path = "UTKFace/UTKFace"
data = []
age = []
for img in os.listdir(path):
  ages = img.split("_")[0]
  img = cv2.imread(str(path)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  data.append(np.array(img))
  age.append(np.array(ages))
age = np.array(age,dtype=np.int64)
data = np.array(data)


In [91]:
print(data.shape)
print(age.shape)

(23708, 200, 200, 3)
(23708,)


In [92]:
#split data to 90/10 to training and testing
x_train,x_test,y_train,y_test = train_test_split(data,age, test_size=0.1, random_state=42)


In [93]:

input = Input(shape=(200,200,3))
conv1 = Conv2D(16,(2,2),activation="relu")(input)
pool1 = MaxPool2D((2,2))(conv1)
batch1 = BatchNormalization()(pool1)
Dropout1 = Dropout(0.5)(batch1)

conv2 = Conv2D(32,(2,2),activation="relu")(Dropout1)
pool2 = MaxPool2D((2,2))(conv2)
batch2 = BatchNormalization()(pool2)
Dropout2 = Dropout(0.5)(batch2)


conv3 = Conv2D(64,(2,2),activation="relu")(Dropout2)
pool3 = MaxPool2D((2,2))(conv3)
batch3 = BatchNormalization()(pool3)
Dropout3 = Dropout(0.5)(batch3)

flt = Flatten()(batch3)
age_l = Dense(64,activation="relu")(flt)
age_l = Dense(32,activation="relu")(age_l)
age_l = Dense(16,activation="relu")(age_l)
age_l = Dense(1,activation="sigmoid")(age_l)

model = Model(inputs=input,outputs= age_l)
model.summary()


Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 199, 199, 16)      208       
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 99, 99, 16)        0         
_________________________________________________________________
batch_normalization_63 (Batc (None, 99, 99, 16)        64        
_________________________________________________________________
dropout_52 (Dropout)         (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 98, 98, 32)        2080      
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 49, 49, 32)        0  

In [ ]:
model.compile(optimizer="adam",loss="mse",metrics=['mae'])
model_CNN  = model.fit(x_train, y_train,
                        validation_split = 0.1,
                        batch_size = 128,
                        epochs=100)
model.save("Agemodel1.h5")

In [ ]:
# Test model
test_model = model.evaluate(x_test, y_test)

In [ ]:
model.save("modelAgeregression1.h5")

In [ ]:
from sklearn.metrics import mean_absolute_error
predicted = model.predict(x_test)
mse=mean_absolute_error(y_test, predicted)